In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(75, 75, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
print('model model')


model model


In [17]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=233):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4,rnd=42)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.48123, saving model to best_m.h5
 - 3s - loss: 0.6565 - acc: 0.5728 - val_loss: 0.4812 - val_acc: 0.7781
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 1s - loss: 0.5602 - acc: 0.6962 - val_loss: 0.5132 - val_acc: 0.7756
Epoch 3/100
Epoch 00003: val_loss improved from 0.48123 to 0.38261, saving model to best_m.h5
 - 1s - loss: 0.4984 - acc: 0.7789 - val_loss: 0.3826 - val_acc: 0.8279
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 1s - loss: 0.4592 - acc: 0.7978 - val_loss: 0.5646 - val_acc: 0.7431
Epoch 5/100
Epoch 00005: val_loss improved from 0.38261 to 0.28504, saving model to best_m.h5
 - 1s - loss: 0.4227 - acc: 0.8078 - val_loss: 0.2850 - val_acc: 0.8753
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 1s - loss: 0.4013 - acc: 0.8258 - val_loss: 0.2982 - val_acc: 0.8554
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 1s - loss: 0.4202 - acc: 0.8103 - val_loss: 0.3353 - val_acc: 0.8579
Epoch 8/100
Epoch 

Epoch 64/100
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.2130 - acc: 0.9142 - val_loss: 0.1826 - val_acc: 0.9127
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.2210 - acc: 0.9067 - val_loss: 0.1879 - val_acc: 0.9202
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.2023 - acc: 0.9092 - val_loss: 0.1805 - val_acc: 0.9227
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.1961 - acc: 0.9092 - val_loss: 0.1942 - val_acc: 0.9152
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.2013 - acc: 0.9106 - val_loss: 0.1791 - val_acc: 0.9202
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.2237 - acc: 0.9042 - val_loss: 0.1827 - val_acc: 0.9227
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.2202 - acc: 0.8954 - val_loss: 0.1910 - val_acc: 0.9227
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 1s - loss: 0.1913 - acc: 0.9233 - val_loss: 0.1817 - val_acc: 0.9177
Epoch 72/100
Epo

Epoch 26/100
Epoch 00026: val_loss improved from 0.26267 to 0.25260, saving model to best_m.h5
 - 1s - loss: 0.2492 - acc: 0.8883 - val_loss: 0.2526 - val_acc: 0.8853
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 1s - loss: 0.2320 - acc: 0.8998 - val_loss: 0.2987 - val_acc: 0.8554
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 1s - loss: 0.2355 - acc: 0.8991 - val_loss: 0.2650 - val_acc: 0.8778
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 1s - loss: 0.2460 - acc: 0.8900 - val_loss: 0.2664 - val_acc: 0.8878
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 1s - loss: 0.2688 - acc: 0.8758 - val_loss: 0.2558 - val_acc: 0.8778
Epoch 31/100
Epoch 00031: val_loss improved from 0.25260 to 0.24888, saving model to best_m.h5
 - 1s - loss: 0.2318 - acc: 0.9031 - val_loss: 0.2489 - val_acc: 0.8878
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 1s - loss: 0.2141 - acc: 0.9108 - val_loss: 0.2493 - val_acc: 0.8803
Epoch 33/100
Epoch 00033: val_loss improved from 0.

Epoch 89/100
Epoch 00089: val_loss did not improve
 - 1s - loss: 0.1770 - acc: 0.9217 - val_loss: 0.2395 - val_acc: 0.8928
Epoch 90/100
Epoch 00090: val_loss improved from 0.22173 to 0.22164, saving model to best_m.h5
 - 1s - loss: 0.1907 - acc: 0.9223 - val_loss: 0.2216 - val_acc: 0.8953
Epoch 91/100
Epoch 00091: val_loss improved from 0.22164 to 0.22066, saving model to best_m.h5
 - 1s - loss: 0.1684 - acc: 0.9267 - val_loss: 0.2207 - val_acc: 0.8928
Epoch 92/100
Epoch 00092: val_loss improved from 0.22066 to 0.21896, saving model to best_m.h5
 - 1s - loss: 0.1887 - acc: 0.9164 - val_loss: 0.2190 - val_acc: 0.9002
Epoch 93/100
Epoch 00093: val_loss improved from 0.21896 to 0.21837, saving model to best_m.h5
 - 1s - loss: 0.1583 - acc: 0.9317 - val_loss: 0.2184 - val_acc: 0.8953
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 1s - loss: 0.1643 - acc: 0.9300 - val_loss: 0.2243 - val_acc: 0.9002
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 1s - loss: 0.1783 - acc: 0.9242 

Epoch 48/100
Epoch 00048: val_loss did not improve
 - 1s - loss: 0.1925 - acc: 0.9167 - val_loss: 0.2346 - val_acc: 0.9127
Epoch 49/100
Epoch 00049: val_loss did not improve
 - 1s - loss: 0.2009 - acc: 0.9142 - val_loss: 0.2313 - val_acc: 0.9077
Epoch 50/100
Epoch 00050: val_loss did not improve
 - 1s - loss: 0.2228 - acc: 0.9106 - val_loss: 0.2299 - val_acc: 0.9027
Epoch 51/100
Epoch 00051: val_loss did not improve
 - 1s - loss: 0.2093 - acc: 0.9133 - val_loss: 0.2290 - val_acc: 0.9002
Epoch 52/100
Epoch 00052: val_loss did not improve
 - 1s - loss: 0.2061 - acc: 0.9100 - val_loss: 0.2434 - val_acc: 0.8978
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 1s - loss: 0.2054 - acc: 0.9092 - val_loss: 0.2303 - val_acc: 0.9027
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 1s - loss: 0.1951 - acc: 0.9208 - val_loss: 0.2323 - val_acc: 0.9052
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 1s - loss: 0.2128 - acc: 0.9050 - val_loss: 0.2234 - val_acc: 0.9052
Epoch 56/100
Epo

Epoch 12/100
Epoch 00012: val_loss did not improve
 - 1s - loss: 0.2733 - acc: 0.8783 - val_loss: 0.4010 - val_acc: 0.8379
Epoch 13/100
Epoch 00013: val_loss improved from 0.32868 to 0.31092, saving model to best_m.h5
 - 1s - loss: 0.3160 - acc: 0.8714 - val_loss: 0.3109 - val_acc: 0.8753
Epoch 14/100
Epoch 00014: val_loss improved from 0.31092 to 0.30401, saving model to best_m.h5
 - 1s - loss: 0.2648 - acc: 0.8850 - val_loss: 0.3040 - val_acc: 0.8853
Epoch 15/100
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.2678 - acc: 0.8781 - val_loss: 0.3047 - val_acc: 0.8903
Epoch 16/100
Epoch 00016: val_loss improved from 0.30401 to 0.30313, saving model to best_m.h5
 - 1s - loss: 0.2529 - acc: 0.8989 - val_loss: 0.3031 - val_acc: 0.8778
Epoch 17/100
Epoch 00017: val_loss did not improve
 - 1s - loss: 0.2604 - acc: 0.9008 - val_loss: 0.3055 - val_acc: 0.8753
Epoch 18/100
Epoch 00018: val_loss improved from 0.30313 to 0.29110, saving model to best_m.h5
 - 1s - loss: 0.2379 - acc: 0.8983 

Epoch 76/100
Epoch 00076: val_loss did not improve
 - 1s - loss: 0.1592 - acc: 0.9275 - val_loss: 0.2734 - val_acc: 0.9027
Epoch 77/100
Epoch 00077: val_loss did not improve
 - 1s - loss: 0.1650 - acc: 0.9214 - val_loss: 0.2778 - val_acc: 0.9052
Epoch 78/100
Epoch 00078: val_loss did not improve
 - 1s - loss: 0.1587 - acc: 0.9308 - val_loss: 0.2805 - val_acc: 0.8953
Epoch 79/100
Epoch 00079: val_loss did not improve
 - 1s - loss: 0.1729 - acc: 0.9275 - val_loss: 0.2754 - val_acc: 0.9077
Epoch 80/100
Epoch 00080: val_loss did not improve
 - 1s - loss: 0.1620 - acc: 0.9217 - val_loss: 0.2893 - val_acc: 0.9052
Epoch 81/100
Epoch 00081: val_loss did not improve
 - 1s - loss: 0.1574 - acc: 0.9308 - val_loss: 0.2830 - val_acc: 0.9002
Epoch 82/100
Epoch 00082: val_loss did not improve
 - 1s - loss: 0.1619 - acc: 0.9292 - val_loss: 0.2829 - val_acc: 0.9002
Epoch 83/100
Epoch 00083: val_loss did not improve
 - 1s - loss: 0.1685 - acc: 0.9325 - val_loss: 0.2697 - val_acc: 0.9077
Epoch 84/100
Epo

In [18]:
import pickle
with open('../features/cnn_2_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_2_aug1_sub.csv', index=False)
# pre 2249
# new 2166

# pre xbg 15195
# rm vertical,shift, fold4, batch 16, 15320, 0.235
# change to 50 epoch, use init setting, 15408, 0.217
# fold3, change to 50 epoch, use init setting, 15436, 0.213
# fold5, 100 epo, 15330, 0.209

# init setting
# rnd 1, 15161, 0.221
# rnd 9, 15383, 0.221
# rnd 42, 15402, 0.218

0.218214012837
         id  is_iceberg
0  5941774d    0.047048
1  4023181e    0.666349
2  b20200e4    0.153661
3  e7f018bb    0.996730
4  4371c8c3    0.221408
